In [22]:
# Challenge 1

# read in the data:
import pandas as pd
df_1 = pd.read_csv('data files/turnstile_190706.txt')
df_2 = pd.read_csv('data files/turnstile_190713.txt')
df_3 = pd.read_csv('data files/turnstile_190720.txt')
df_4 = pd.read_csv('data files/turnstile_190727.txt')
df_5 = pd.read_csv('data files/turnstile_190803.txt')
df_6 = pd.read_csv('data files/turnstile_190810.txt')
df_7 = pd.read_csv('data files/turnstile_190817.txt')
df_8 = pd.read_csv('data files/turnstile_190824.txt')
df_9 = pd.read_csv('data files/turnstile_190831.txt')

#concatenate
df_combined = pd.concat([df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9])

In [23]:
# Column name 'EXITS' needs to be formatted to remove extra spaces
print(df_combined.columns)
df_combined.rename(columns={df_combined.columns[10]:'EXITS'}, inplace=True)
print(df_combined.columns)

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES',
       'EXITS                                                               '],
      dtype='object')
Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES', 'EXITS'],
      dtype='object')


In [24]:
# Challenge 2

# What data type did pandas assign values in columns 'Date' and 'Time'?
print(type(df_combined.DATE.iloc[0]), type(df_combined.TIME.iloc[0]))

# Combine 'Date' and 'Time' columns and format into a datetime object
df_combined['DATETIME'] = pd.to_datetime(df_combined['DATE']+ ' ' + df_combined['TIME'])
#df_combined.drop(columns=['DATE','TIME'], inplace=True, axis=1)
print(df_combined.head())

<class 'str'> <class 'str'>
    C/A  UNIT       SCP STATION LINENAME DIVISION        DATE      TIME  \
0  A002  R051  02-00-00   59 ST  NQR456W      BMT  06/29/2019  00:00:00   
1  A002  R051  02-00-00   59 ST  NQR456W      BMT  06/29/2019  08:00:00   
2  A002  R051  02-00-00   59 ST  NQR456W      BMT  06/29/2019  12:00:00   
3  A002  R051  02-00-00   59 ST  NQR456W      BMT  06/29/2019  16:00:00   
4  A002  R051  02-00-00   59 ST  NQR456W      BMT  06/29/2019  20:00:00   

      DESC  ENTRIES    EXITS            DATETIME  
0  REGULAR  7116734  2410183 2019-06-29 00:00:00  
1  REGULAR  7116770  2410218 2019-06-29 08:00:00  
2  REGULAR  7116856  2410288 2019-06-29 12:00:00  
3  REGULAR  7117076  2410346 2019-06-29 16:00:00  
4  REGULAR  7117340  2410387 2019-06-29 20:00:00  


Challenge 3

Each row is a turnstile, identified by a combination of the C/A, UNIT, SCP, and STATION columns, with information on entries and exits at that turnstile every n hours. (What is n?) We want total daily entries.
Group the data so that it represents daily entries for each turnstile (hint: pd.groupby or DataFrame.groupby)

In [21]:
df_combined['TURNSTILES'] = (df_combined['STATION'] + '_' + df_combined['C/A'] + '_' + df_combined['UNIT'] + '_' + df_combined['SCP'])

turnstiles_df = turnstiles_df.sort_values(by=['DATETIME'])

turnstiles_df = df_combined.groupby(["TURNSTILES", "DATETIME", "TIME"]).sum().diff(axis=0).fillna(0)

KeyError: 'TIME'

In [ ]:
# FILTER FOR TIME BEFORE COMBINING INTO GROUPS

In [41]:
df_combined.head(10)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME,hour
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/29/2019,00:00:00,REGULAR,7116734,2410183,2019-06-29 00:00:00,0
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/29/2019,08:00:00,REGULAR,7116770,2410218,2019-06-29 08:00:00,8
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/29/2019,12:00:00,REGULAR,7116856,2410288,2019-06-29 12:00:00,12
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/29/2019,16:00:00,REGULAR,7117076,2410346,2019-06-29 16:00:00,16
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/29/2019,20:00:00,REGULAR,7117340,2410387,2019-06-29 20:00:00,20
5,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/30/2019,00:00:00,REGULAR,7117480,2410406,2019-06-30 00:00:00,0
6,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/30/2019,04:00:00,REGULAR,7117508,2410415,2019-06-30 04:00:00,4
7,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/30/2019,08:00:00,REGULAR,7117523,2410431,2019-06-30 08:00:00,8
8,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/30/2019,12:00:00,REGULAR,7117610,2410486,2019-06-30 12:00:00,12
9,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/30/2019,16:00:00,REGULAR,7117803,2410531,2019-06-30 16:00:00,16


In [29]:
df_combined["hour"] = df_combined.DATETIME.dt.hour

In [53]:
hour_filter = (df_combined['hour'] < 21) & (df_combined['hour'] > 6)
filtered_df = df_combined[hour_filter]

In [54]:
filtered_df.head(10)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME,hour
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/29/2019,08:00:00,REGULAR,7116770,2410218,2019-06-29 08:00:00,8
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/29/2019,12:00:00,REGULAR,7116856,2410288,2019-06-29 12:00:00,12
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/29/2019,16:00:00,REGULAR,7117076,2410346,2019-06-29 16:00:00,16
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/29/2019,20:00:00,REGULAR,7117340,2410387,2019-06-29 20:00:00,20
7,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/30/2019,08:00:00,REGULAR,7117523,2410431,2019-06-30 08:00:00,8
8,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/30/2019,12:00:00,REGULAR,7117610,2410486,2019-06-30 12:00:00,12
9,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/30/2019,16:00:00,REGULAR,7117803,2410531,2019-06-30 16:00:00,16
10,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/30/2019,20:00:00,REGULAR,7118006,2410568,2019-06-30 20:00:00,20
13,A002,R051,02-00-00,59 ST,NQR456W,BMT,07/01/2019,08:00:00,REGULAR,7118170,2410688,2019-07-01 08:00:00,8
14,A002,R051,02-00-00,59 ST,NQR456W,BMT,07/01/2019,12:00:00,REGULAR,7118351,2410859,2019-07-01 12:00:00,12


In [55]:
#filtered_df['TURNSTILES'] = (filtered_df['STATION'] + '_' + filtered_df['C/A'] + '_' + filtered_df['UNIT'] + '_' + filtered_df['SCP'])
#filtered_df = filtered_df.sort_values(by=['C/A','UNIT','SCP','STATION','DATETIME'])
filtered_df = filtered_df.groupby(["STATION","hour"]).sum().fillna(0)





In [56]:
filtered_df.head()

ENTRIES         EXITS
STATION hour                            
1 AV    8     108472515171  112360330010
        9        454740047     443774718
        10       454839881     443844942
        12     91485568372   94876713440
        16     89125025934   92297277187

In [25]:








turnstiles_df['TOTAL'] = turnstiles_df['ENTRIES'] + turnstiles_df['EXITS']

KeyError: 'TURNSTILES'

In [12]:
turnstiles_df.drop(columns=['ENTRIES','EXITS'], inplace=True)

In [15]:
sample_df = turnstiles_df.sample(1000, random_state=1)

In [20]:
sample_df.head()

,,TOTAL
TURNSTILES,DATETIME,
BEACH 98 ST_N185_R417_00-00-02,2019-07-24 17:00:00,114.0
NOSTRAND AV_N117_R198_01-00-00,2019-07-08 12:00:00,672.0
FOREST AVE_K024_R403_00-00-00,2019-07-29 09:00:00,466.0
WALL ST_R112_R027_02-00-00,2019-07-17 00:00:00,54.0
CENTRAL PK N110_R301_R323_00-00-00,2019-08-24 00:00:00,130.0


In [19]:
# find busiest hour within subset of hours (times < 06:00 and times >21:00)
sample_df["hour"] = sample_df.DATETIME.dt.hour


#sample_df[''] = 
#mask = sample_df['DATETIME'] > 06:00
#filtered_df = sample_df[]



AttributeError: 'DataFrame' object has no attribute 'DATETIME'

In [18]:
sample_df.head(20)

,,TOTAL
TURNSTILES,DATETIME,
BEACH 98 ST_N185_R417_00-00-02,2019-07-24 17:00:00,114.0
NOSTRAND AV_N117_R198_01-00-00,2019-07-08 12:00:00,672.0
FOREST AVE_K024_R403_00-00-00,2019-07-29 09:00:00,466.0
WALL ST_R112_R027_02-00-00,2019-07-17 00:00:00,54.0
CENTRAL PK N110_R301_R323_00-00-00,2019-08-24 00:00:00,130.0
BOWLING GREEN_R200A_R041_01-00-02,2019-07-31 04:00:00,15.0
BAY RIDGE-95 ST_C027_R216_00-03-00,2019-08-25 00:00:00,91.0
46 ST_N316A_R267_01-06-00,2019-08-10 13:00:00,126.0
HARRISON_PTH02_R544_00-06-01,2019-07-19 10:01:56,13.0


In [ ]:
# find busiest hour within subset of hours (times < 06:00 and times >21:00)

# Sum Entries and Exits

# Do we still have high negative and positive counts?
#   are there ways to clean this data?

#
#   Step 1 what are the busiest stations?
#

# group by station, get sum
station_df = df_combined.groupby(["STATION", "DATETIME"]).sum().diff(axis=0).fillna(0)

# group by station, get mean
station_df = df_combined.groupby(["STATION", "DATETIME"]).mean().diff(axis=0).fillna(0)

# look at SD to understand how variable values are at each station



# bar chart - all
# bar chart - top ~30?




#
#   Step 2 what stations are most important to the client?
#

# Census data: residential/ tourist/ income

# Visualize relevant neighborhoods with NYC map

# bar chart - top ~30 highlight/ gray-out

# conclusions about recommended stations for canvasing



#
#   Step 3 break down stations by busiest hour to refine recommendation about when to canvas
#

# focus on the short list of prioritized stations

# write OR statement to include times in 8a-12p and 9a-1p (8a-1p)















In [ ]:
turnstiles_df = turnstiles_df.reset_index()
mask = (turnstiles_df['TURNSTILES'] == 'FLATBUSH AV-B.C_R646_R110_01-00-01')
fb_df = turnstiles_df[mask]


In [ ]:
(fb_df['ENTRIES'] < 0).value_counts()

In [ ]:
data = [['A','1'],['B','2'],['B','3'],['A','2'],['B','1'],['A','3']]
df = pd.DataFrame(data, columns=['station','scp'])
df
df.sort_values(by=['station','scp'], inplace=True)
df

### A look at .diff() on a SUBSET of grouped values  

In [ ]:
tmp2 = df_combined[0:300]
tmp2['TURNSTILES'] = (tmp2['STATION'] + '_' + tmp2['C/A'] + '_' + tmp2['UNIT'] + '_' + tmp2['SCP'])
tmp2_group = tmp2.groupby(["TURNSTILES", "DATE"]).sum().diff(axis=0)
tmp2_group.head(40)

### Look at resetting index first...working

In [ ]:
"""tmp2 = df_combined
tmp2['TURNSTILES'] = (tmp2['STATION'] + '_' + tmp2['C/A'] + '_' + tmp2['UNIT'] + '_' + tmp2['SCP'])
tmp2_group = tmp2.groupby(["TURNSTILES", "DATE"]).sum().reset_index()
tmp2_group.head(50)
#diff(axis=0)."""

### A look at .diff() on ALL grouped values  (THIS IS CURRENT WORKFLOW)

In [ ]:
tmp2 = df_combined
tmp2['TURNSTILES'] = (tmp2['STATION'] + '_' + tmp2['C/A'] + '_' + tmp2['UNIT'] + '_' + tmp2['SCP'])
tmp2_group = tmp2.groupby(["TURNSTILES", "DATE"]).sum().diff(axis=0).fillna(0)
tmp2_group.head(50)

### A Look at .diff() on un-grouped values  
  
Most values are okay. We get large negatives (and probably large positives) when we take the difference between different turnstiles...note row 40 vs 41.  
  
Maybe we can just filter out those values. The data we're losing are entry/ exit counts between 8:00pm and midnight (in this example). Those times may be different in other cases, but likely occur at 'overnight' times.  
  
Maybe if we sort all the data first by C/A, UNIT, SCP, STATION, then DATETIME, maybe we can cut down on the number of occurrences of these errors. 

In [ ]:
tmp = df_combined[30:50]
tmp['ENT_DIFF'] = tmp['ENTRIES'].diff()
tmp